# 🤖 Building Autonomous Agents: Exploring Agent Frameworks:

In this module, we'll examine how different agent frameworks implement autonomous agents, focusing specifically on LangChain/LangGraph, PydanticAI, and CrewAI. We'll explore how these frameworks handle orchestration, tool use, and agent coordination while leveraging our existing abstractions.

Objectives:
* Get hands on with high-level frameworks like LangChain/LangGraph, PydanticAI, and CrewAI
* Learn how to integrate our tool calling, memory, and conversation abstractions with each framework
* Implement examples showing how to maintain consistent interfaces across frameworks
* Understand when to use each framework based on their strengths and application needs

By the end of this module, you'll understand how to build on top of these frameworks while reusing your existing code, allowing you to choose the right framework for each use case without starting from scratch.

# LangChain
We've already explored LangGraph a bit in module 2, but we haven't spent much time with LangChain. In this section, we'll be using the langchain-aws repo to play with LangChain on Bedrock. We'll try to accomplish 3 things
* Discuss abstraction layers
* Discuss pros/cons
* Recreate the agent we built in the previous notebooks

First lets start with a very simple chat

In [ ]:
# Initialize the Bedrock chat model
from langchain_aws import ChatBedrockConverse
from langchain_core.messages import AIMessage, BaseMessage

llm = ChatBedrockConverse(
    model="us.anthropic.claude-3-sonnet-20240229-v1:0",
    temperature=0
)

# Invoke the llm
messages = [
    ("system", "You are a helpful assistant."),
    ("human", "Hello! How are you today?"),
]

response: AIMessage =llm.invoke(messages)
print(response)

Getting LangChain to work out of the box is very simple. Now lets recreate the agent we built in the previous lab with LangChain.

First lets reuse our tools from the previous lab. We can leverage a nice abstraction LangGraph provides called create_react_agent() to take these tools and quickly create an agent. 

In essence, this abstracts away a lot of the work we did in the previous notebook to build a ReACT like agent! One cool thing about frameworks is that they usually take in "callable's" meaning you can just pass in a function with a doc string and it'll work. 

In [ ]:
from typing import Dict, Any, List, Callable

# Import our tools from the previous lab.
from agentic_platform.core.tool.sample_tools import weather_report, handle_calculation

from langgraph.prebuilt import create_react_agent
from langgraph.graph import Graph

# Use the prebuilt react agent from LangGraph
agent: Graph = create_react_agent(model=llm, tools=[weather_report, handle_calculation])

# Invoke the agent
inputs = {"messages": [("user", "What's the weather in San Francisco?")]}
response = agent.invoke(inputs)
# Print the response
for message in response['messages']:
    print(message)

Awesome! In just a couple lines of code were able to recreate the work we did in a previous 3 labs!

## A Word on Lock-in. 
Using LangGraph / LangChain (or any framework) makes sense in scenarios like this. It does exactly what we need it to do out of the box. However, it has it's own types (messages), it's own model invocation implementation, etc.. You can use them, but it creates a 1-way door decision. If you need to build something custom, use different framework, swap out a long term memory implementation, etc.. it will be very painful to undo the tight coupling of a framework.

To solve this, we just need to wrap the code above into our own types and then the rest of the system doesn't care what framework you're using. It's a little extra work but provides a lot more flexibility. As things become more standard, this might become less of a problem. But for now, the best way to create 2-way doors is to use your own types. 

Now lets wrap the agent above in our own abstractions to create interoperability.

In [ ]:
# This is undifferentiated converter code so we pushed it to the common folder.
# LangChains message format differs significantly from other model APIs so we had to take some short cuts.
# Ex) Tool Calls are converted into strings even though it's a pydantic model. 
# LangChain also has the concept of a tool message which some providers use but others don't.
# This is why we have a converter.
from agentic_platform.core.converter.langchain_converters import LangChainMessageConverter

Now lets build our agent and use our abstractions to create interoperability between our custom agent and LangChain

In [ ]:
from agentic_platform.core.models.api_models import AgenticRequest, AgenticResponse
from agentic_platform.core.models.prompt_models import BasePrompt
from agentic_platform.core.models.memory_models import Message, SessionContext
from typing import Dict, Any, List, Callable

# Lets reuse our memory client from the previous lab.
# Clients.
class MemoryClient:
    """Manages conversations"""
    def __init__(self):
        self.conversations: Dict[str, SessionContext] = {}

    def upsert_conversation(self, conversation: SessionContext) -> bool:
        self.conversations[conversation.session_id] = conversation

    def get_or_create_conversation(self, conversation_id: str=None) -> SessionContext:
        return self.conversations.get(conversation_id, SessionContext()) if conversation_id else SessionContext()

from langchain_core.tools import Tool
from langgraph.prebuilt import create_react_agent
from langgraph.graph import Graph

memory_client: MemoryClient = MemoryClient()

class LangChainAgent:
    
    def __init__(self, tools: List[Callable], base_prompt: BasePrompt):
        # Do some conversions to take our types and make them work with LangChain.
        temp: float = base_prompt.hyperparams["temperature"] if "temperature" in base_prompt.hyperparams else 0.5
        llm: ChatBedrockConverse = ChatBedrockConverse(
            model=base_prompt.model_id,
            temperature=temp
        )

        # We'll use a prebuilt graph from langgraph that implements the same React pattern.
        # This should be done at instantiation time to reduce the overhead of re-compiling the graph.
        self.agent: Graph = create_react_agent(model=llm, tools=tools)
        self.conversation: SessionContext = None

    def invoke(self, request: AgenticRequest) -> AgenticResponse:
        # Get or create conversation
        self.conversation = memory_client.get_or_create_conversation(request.session_id)
        # Add user message to conversation
        self.conversation.add_message(request.message)
        # Convert to langchain messages
        inputs = {"messages": [("user", request.message.text)]}
        response = self.agent.invoke(inputs)
        print(response['messages'])
        # Convert to our response format
        messages: List[Message] = LangChainMessageConverter.convert_langchain_messages(response['messages'])
        # Add messages to conversation
        self.conversation.add_messages(messages)
        # Save the conversation
        memory_client.upsert_conversation(self.conversation)
        # Return the response
        return AgenticResponse(
            session_id=self.conversation.session_id,
            message=self.conversation.messages[-1]
        )

In [ ]:
from agentic_platform.core.tool.sample_tools import weather_report, handle_calculation

# Define our agent prompt.
class AgentPrompt(BasePrompt):
    system_prompt: str = '''You are a helpful assistant.'''
    user_prompt: str = '''{user_message}'''

# Build out our prompt
user_message: str = 'What is the weather in San Francisco?'
prompt: BasePrompt = AgentPrompt()
# Instantiate the agent

tools: List[Callable] = [weather_report, handle_calculation]
my_agent: LangChainAgent = LangChainAgent(base_prompt=prompt, tools=tools) 

# Create the agent request. Same as our other agent type in the tool calling lab.
request: AgenticRequest = AgenticRequest.from_text(text=user_message)

# Invoke the agent
response: AgenticResponse = my_agent.invoke(request)

print(response.message)

In [ ]:
# Now lets look at our conversation.
conversation: SessionContext = memory_client.get_or_create_conversation(response.session_id)
# Use the pydantic model_dump_json
print(conversation.model_dump_json(indent=2, serialize_as_any=True))

# Now we have interoperability!
By adding some converters and wrapping LangChain/LangGraph in our own abstractions, we were able to 
1. Return the same agent response as our custom agent so we can reuse AgentRequest & AgentResponse types
2. Have a universal memory implementation across different agents built with different frameworks
3. Decoupled ourselves from relying too much on any framework, API provider, etc.. 

By owning our own types, we can begin to see how you can mix and match frameworks to get the best of both worlds while still maintaining control over your system. 



# Pydantic AI
Pydantic AI is a newer framework. The main draw is type safety. you can create type safe graphs and build agents relatively quickly. It also works really well with pydantic models. 

In the examples with LangChain above, the react agent can't handle the nested pydantic objects of the calculator function and will usually error out unless you change your function definition. PydanticAI doesn't have that problem.

It is fairly new (as of 4/20/2025) but is a framework worth watching

In [ ]:
# First we need to use nest_asyncio which patches the asyncio to allow nested event loops which PydanticAI runs on.
import nest_asyncio
nest_asyncio.apply()


In [ ]:
# First Lets create a simple agent. We need to start aliasing the agent class to avoid conflict with the langchain agent.
from pydantic_ai import Agent as PyAIAgent

pyai_agent: PyAIAgent = PyAIAgent(
    'bedrock:anthropic.claude-3-sonnet-20240229-v1:0',
    system_prompt='You are a helpful assistant.',
)

# Now lets add our existing tools to the agent. Notice how the tool object actually lives on the agent object itself. 
# Secondly, PydanticAI has two types of tools. tool() has access to the run context while tool_plain() does not.
# We'll use the plain tool here since we don't need access to the run context.
tools: List[Tool] = [pyai_agent.tool_plain(func)for func in [weather_report, handle_calculation]]

In [ ]:
result_sync = pyai_agent.run_sync('What is 7 plus 7?')
print(result_sync.data)

print("--------------------------------")

for message in result_sync.all_messages():
    print(message)

# Create our Abstraction Layers
Pydantic AI has a pretty clean design around agents. However, we still want to own our own types to make it interoperable with other parts of our system. Lets create our wrappers and converters like with did with langchain

In [ ]:
# Like the langchain converter, this is undifferentiated code so we pushed it to the common folder.
from agentic_platform.core.converter.pydanticai_converters import PydanticAIMessageConverter

messages: List[Message] = PydanticAIMessageConverter.convert_messages(result_sync.all_messages())

for message in messages:
    print(message)

# Rewrite Our Agent in Pydantic
Now we can rewrite our agent in Pydantic. We'll reuse the same memory client to show how we can store conversations across various frameworks / models without having to rewrite our code. 

In [ ]:
from pydantic_ai import Agent as PyAIAgent
from pydantic_ai.models import ModelResponse

class PydanticAIAgent:
    
    def __init__(self, tools: List[Callable], base_prompt: BasePrompt):
        # This is the identifier for PydanticAI calling Bedrock.
        model_id = f'bedrock:{base_prompt.model_id}'
        self.agent: PyAIAgent = PyAIAgent(
            model_id,
            system_prompt=base_prompt.system_prompt,
        )

        # Add our tools to the agent.
        [self.agent.tool_plain(func)for func in tools]

    def invoke(self, request: AgenticRequest) -> AgenticResponse:
        # Get or create conversation
        conversation: SessionContext = memory_client.get_or_create_conversation(request.session_id)
        # Convert to langchain messages
        response: ModelResponse = self.agent.run_sync(request.message.text)
        # Convert to our response format
        messages: List[Message] = PydanticAIMessageConverter.convert_messages(response.all_messages())
        # Add messages to conversation
        conversation.add_messages(messages)
        # Save the conversation
        memory_client.upsert_conversation(conversation)
        # Return the response
        return AgenticResponse(
            session_id=conversation.session_id,
            message=messages[-1]
        )

In [ ]:
# Define our agent prompt.
class AgentPrompt(BasePrompt):
    system_prompt: str = '''You are a helpful assistant.'''
    user_prompt: str = '''{user_message}'''

# Build out our prompt
user_message: str = 'What is the weather in San Francisco?'
prompt: BasePrompt = AgentPrompt()
# Instantiate the agent
my_agent: PydanticAIAgent = PydanticAIAgent(base_prompt=prompt, tools=tools) 

# Create the agent request. Same as our other agent type in the tool calling lab.
request: AgenticRequest = AgenticRequest.from_text(text=user_message)

# Invoke the agent
response: AgenticResponse = my_agent.invoke(request)

print(response.message.model_dump_json(indent=2))

In [ ]:
# Now lets look at our conversation.
conversation: SessionContext = memory_client.get_or_create_conversation(response.session_id)
# Use the pydantic model_dump_json
print(conversation.model_dump_json(indent=2, serialize_as_any=True))

# Conclusion
This concludes module 3 on autonomous agents. In this lab we:
1. Explored 2 of the many agent frameworks available today
2. Demonstrated how to make agent frameworks interoperable and create 2 way door decisions with proper abstraction in code. 

In the next module we'll be discussing some more advanced concepts of agents. Specifically multi-agent systems and model context protocol (MCP)